In [0]:
from pyspark.sql.functions import *

#Reading Hospital A providers data 
df_hosa=spark.read.parquet("abfss://bronze@databricksdevetl.dfs.core.windows.net/hosa/providers")

#Reading Hospital B providers data 
df_hosb=spark.read.parquet("abfss://bronze@databricksdevetl.dfs.core.windows.net/hosb/providers")

#union two hospitals dataframes
df_merged = df_hosa.unionByName(df_hosb)
display(df_merged)

df_merged.createOrReplaceTempView("providers")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_dev.silver.providers (
  ProviderID string,
  FirstName string,
  LastName string,
  Specialization string,
  DeptID string,
  NPI long,
  datasource string,
  is_quarantined boolean
)
USING DELTA;

In [0]:
%sql
TRUNCATE TABLE databricks_dev.silver.providers;

In [0]:
%sql 
INSERT INTO TABLE databricks_dev.silver.providers
SELECT DISTINCT
  ProviderID,
  FirstName,
  LastName,
  Specialization,
  DeptID,
  try_cast(NPI as INT) NPI,
  datasource,
      CASE 
          WHEN ProviderID IS NULL OR DeptID IS NULL THEN TRUE
          ELSE FALSE
      END AS is_quarantined
FROM providers;

In [0]:
%sql
SELECT * FROM providers;